# Getting background information:
---

- New York City is divided into 5 boroughs, which are essentially distinct regions.
- Schools in New York City are divided into several school district, each of which can contain dozens of schools.
- Not all the schools in all of the datasets are high schools, so we’ll need to do some data cleaning.
- Each school in New York City has a unique code called a DBN, or District Borough Number.
- By aggregating data by district, we can use the district mapping data to plot district-by-district differences.


### Process:
---
- Loop through each data file we downloaded.
- Read the file into a Pandas DataFrame.
- Put each DataFrame into a Python dictionary.


In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/home/harsh/anaconda2/lib/python2.7/site-packages/numexpr/cpuinfo.py:42: UserWarning: [Errno 2] No such file or directory
  warnings.warn(str(e), UserWarning, stacklevel=stacklevel)


In [2]:
files = ['ap_2010.csv','class_size.csv','demographics.csv','graduation.csv','sat_results.csv','math_result.csv','hs_directory.csv']
data={}

for f in files:
    d = pd.read_csv("NYC_SAT_DATASETS/{0}".format(f))
    
    data[f.replace('.csv','')] = d


In [3]:
for key,df in data.items():
    print key,"\n","-"*90
    print df.head(5),"\n","-"*90

sat_results 
------------------------------------------------------------------------------------------
      DBN                                    SCHOOL NAME  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   
3  01M458                      FORSYTH SATELLITE ACADEMY   
4  01M509                        MARTA VALLE HIGH SCHOOL   

  Num of SAT Test Takers SAT Critical Reading Avg. Score SAT Math Avg. Score  \
0                     29                             355                 404   
1                     91                             383                 423   
2                     70                             377                 402   
3                      7                             414                 401   
4                     44                             390                 433   

  SAT Writing Avg. Score  
0                    363  


---
### Some useful patters that can be observed:


- Most of the datasets contain a DBN column
- Some fields look interesting for mapping, particularly Location 1, which contains coordinates inside a larger string.
- Some of the datasets appear to contain multiple rows for each school (repeated DBN values), which means we’ll have to do some preprocessing.
---
### Observations for cleaning :
- We can merge these multiple datasets using the DBN column (DBN is a unique code for each school).
- two of the datasets, class_size, and hs_directory, don’t have a DBN field.
- In the hs_directory data, it’s just named dbn, so we can just rename the column, or copy it over into a new column called DBN.
- In the class_size data, we’ll need to try a different approach.
---
<font size=3>
On close inspection we find that DBN is actually a combination of three columns from class size:<br>
1.CSD <br>
2.BOROUGH<br>
3.SCHOOL CODE<br><br>

Have a look below:
</font>


In [4]:
data['class_size'].head(4)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN


In [5]:
data["math_result"].DBN.head(4)

0    01M015
1    01M015
2    01M015
3    01M015
Name: DBN, dtype: object

---
<font size=4>

Now we know how to construct DBN. We can now use this knowledge to our advantage and create a new column in "class_size" 
</font>

---

In [6]:
# 0:02d the element(0:) formated as an integer (d) to a field of minimum width 2 (2), with zero-padding on the left (leading 0):

f = lambda x: "{0:02d}{1}".format(x["CSD"], x["SCHOOL CODE"])

In [7]:
data['class_size']['DBN'] = data['class_size'].apply(f, axis=1)
data['class_size']['DBN'].head(4)

0    01M015
1    01M015
2    01M015
3    01M015
Name: DBN, dtype: object

---

<font size=3>
We have also found an intresting dataset on student, parent, and teacher surveys about the quality of schools.
These surveys include information about the perceived safety of each school, academic standards, and more. <br><br>
Lets add the survey data into our data dictionary, and then combine all the datasets afterwards. 
The survey data consists of 2 files, one for all schools, and one for school district 75.We will now perform these steps:
</font>

- Read in the surveys for all schools
- Read in the surveys for district 75 school
- Add a flag that indicates which school district each dataset is for.
- Combine the datasets into one using the concat method on DataFrames.
---

**NOTE:** the survey dataset comes as a zip and contains 2 of the survey files as **xlsx** and two as **.txt**.<br>
xlsx will take more than 10 minutes to read so we will use txt files, but these txts are encoded in **windows-1252** format so as to make it faster to read

In [8]:
survey1 = pd.read_csv("NYC_SAT_DATASETS/survey1.txt",encoding='windows-1252',delimiter="\t")
survey2 = pd.read_csv("NYC_SAT_DATASETS/surveyd75.txt",encoding='windows-1252',delimiter="\t")  #district 75

survey1.d75 = False
survey2.d75 = True

survey = pd.concat([survey1,survey2],axis=0)

---
<font size=3>
All the information related to the survey data columns and feilds is given in a "survey data dictionar.xlsx' which comes along with the 4 survey files in the same zip.<br><br>

We can use this file to get an insight on the most important columns so that we can eleminate all the unwanted columns and make analysis possible as there are **2773 columns**.
</font>

---

In [9]:
survey['DBN'] = survey.dbn
columns = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_10", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]
survey = survey.loc[:,columns]

data['survey'] = survey

In [10]:
survey.shape

(1702, 23)

In [11]:
class_size = data['class_size']
columns = [i.strip() for i in class_size.columns]
class_size.columns = columns

In [12]:
print class_size.GRADE.value_counts().head(4), '\n','-'*50
print class_size["PROGRAM TYPE"].value_counts().head(4)

09-12      10644
MS Core     4762
0K-09       1384
0K          1237
Name: GRADE, dtype: int64 
--------------------------------------------------
GEN ED     14545
CTT         7460
SPEC ED     3653
G&T          469
Name: PROGRAM TYPE, dtype: int64


---
<font size=3>
With the class_size dataset, it looks like GRADE and PROGRAM TYPE have multiple values for each school. By restricting each field to a single value, we can filter most of the duplicate rows.We will now perform these steps:
</font>
 
- Only select values from class_size where the GRADE field is 09-12.
- Only select values from class_size where the PROGRAM TYPE field is GEN ED.
- Group the class_size dataset by DBN, and take the average of each column. Essentially, we’ll find the average class_size - values for each school.
- Reset the index, so DBN is added back in as a column.
---

In [13]:
class_size = class_size[class_size.GRADE == '09-12']
class_size = class_size[class_size['PROGRAM TYPE'] == 'GEN ED']
class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)
data['class_size'] = class_size

# Lets now start condensing other datasets:

---
## Demogrpahics:
---
The data for **Demographics dataset** was collected for multiple years for the same schools, so there are duplicate rows for each school. We’ll only pick rows where the schoolyear field is the most recent available.

In [14]:
demo = data['demographics']

In [15]:
demo.head(3)

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2


In [16]:
demo = demo[demo.schoolyear== 20112012]
data['demographics'] = demo

---
# math_result:
---
<font size=3>
We’ll need to condense the math_result dataset. This dataset is segmented by Grade and by Year. We can select only a single grade from a single year:
</font>

In [17]:
data["math_result"] = data["math_result"][data["math_result"]["Year"] == 2011]
data["math_result"] = data["math_result"][data["math_result"]["Grade"] == '8']

---
## graduation:
---
<font size=3>
Finally, graduation needs to be condensed:
</font>

In [18]:


data["graduation"] = data["graduation"][data["graduation"]["Cohort"] == "2006"]
data["graduation"] = data["graduation"][data["graduation"]["Demographic"] == "Total Cohort"]



# Variable Computation:
---
<font size=3>
We will be computing a total SAT score **SAT Math Avg. Score**, **SAT Critical Reading Avg. Score**, and **SAT Writing Avg. Score**. In the below code, we:
</font>

- Convert each of the SAT score columns from a string to a number.
- Add together all of the columns to get the sat_score column, which is the total SAT score.


In [19]:
columns = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score','SAT Writing Avg. Score']
sat = data["sat_results"]

In [20]:
for c in columns:
    sat[c] = sat[c].convert_objects(convert_numeric = True)

/home/harsh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [21]:
sat['sat_score'] = 0

for c in columns:
    sat['sat_score'] += sat[c]
    
data['sat_results'] = sat

In [22]:
data['sat_results'].head(5)

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,1174.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0


---
## hs_directory:
---
Now we will parse out the coordinate locations of each school, so we can make maps. This will enable us to plot the location of each school. In the below code, we:


- Parse latitude and longitude columns from the Location 1 column.
- Convert lat and lon to be numeric.

In [23]:
hs_dir = data['hs_directory']

hs_dir['lat'] = hs_dir['Location 1'].apply(lambda x: x.split("\n")[2].replace("(",'').replace(')','').split(", ")[0])
hs_dir['lon'] = hs_dir['Location 1'].apply(lambda x: x.split("\n")[2].replace("(",'').replace(')','').split(", ")[1])

In [24]:
for c in ['lat','lon']:
    hs_dir[c] = hs_dir[c].convert_objects(convert_numeric = True)

/home/harsh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [25]:
hs_dir['DBN'] = hs_dir.dbn
hs_dir.drop("dbn",inplace=True,axis=1)

In [26]:
data['hs_directory'] = hs_dir

## Now lets take a look at our work till now :

In [27]:
for key , df in data.items():
    print "-"*70,'\n',key,'\n',"-"*70,'\n\n'
    print df.head(3)

---------------------------------------------------------------------- 
sat_results 
---------------------------------------------------------------------- 


      DBN                                    SCHOOL NAME  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   

  Num of SAT Test Takers  SAT Critical Reading Avg. Score  \
0                     29                            355.0   
1                     91                            383.0   
2                     70                            377.0   

   SAT Math Avg. Score  SAT Writing Avg. Score  sat_score  
0                404.0                   363.0     1122.0  
1                423.0                   366.0     1172.0  
2                402.0                   370.0     1149.0  
---------------------------------------------------------------------- 
demographics 
-----------------------------

## Combining the datasets:
---------------------
<font size=3>
We have performed all the preliminaries... we can finally combine the datasets together using the DBN column.At the end, we’ll have a dataset with hundreds of columns, from each of the original datasets.<br><br>
We will be joining the datasets using the **"OUTER" join** as some of the datasets are missing high schools that exist in the sat_results, so that we dont lose data.
</font>

We will now go through the following process:

- Loop through each of the items in the data dictionary.
- Print the number of non-unique DBNs in the item.
- Decide on a join strategy – inner or outer.
- Join the item to the DataFrame full using the column DBN.



In [28]:
# Separating keys and data:
dataset_names = data.keys()
data_flat = [data[key] for key in dataset_names]

dataset_names[0] ,dataset_names[5] = dataset_names[5], dataset_names[0]
data_flat[0] , data_flat[5] = data_flat[5] , data_flat[0]

In [29]:
full = data_flat[0]
print "\nnon-unique DBN values:\n"

for i , df in enumerate(data_flat[1:]):
    name = dataset_names[i+1]
    
    print name
    print (len(df['DBN'])-len(df['DBN'].unique()))
    
    
    join = "inner"    
    if name in ["sat_results", "ap_2010", "graduation"]:
        join = "outer"
    if name not in ["math_result"]:
        full = full.merge(df, on="DBN", how=join)
    
full.shape


non-unique DBN values:

demographics
0
graduation
0
hs_directory
0
ap_2010
1
sat_results
0
survey
0
class_size
0


(443, 179)

---
<font size =3 >
Now we may want to correlate the Advanced Placement exam results with SAT scores, but we’ll need to first convert those columns to numbers, then fill in any missing values:
</font>

---

In [30]:
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']

for col in cols:
    full[col] = full[col].convert_objects(convert_numeric=True)

full[cols] = full[cols].fillna(value=0)

/home/harsh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


---
<font size=3>
we’ll now calculate a school_dist column that indicates the school district of the school. This will enable us to match up school districts and plot out district-level statistics using the district maps.
</font>

In [71]:
full['school_dist'] = full["DBN"].apply(lambda x: x[:2])

---
<font size =3>
Finally we will now fill in all the missing values in each column od the dataset with mean of each column.
</font>

In [32]:
full = full.fillna(full.mean())

## Computing correlations:
---
<font size=3>
This step will tell us which columns are closely related to the column we’re interested in(i.e."SAT SCORE"). The closer to 0 the correlation, the weaker the connection. The closer to 1, the stronger the positive correlation, and the closer to -1, the stronger the negative correlation:

</font>

In [33]:
#corr = full.corr()['sat_score']
#corr.sort_values(ascending=False)

full.corr()['sat_score'].sort_values(ascending=False)

sat_score                               1.000000
SAT Writing Avg. Score                  0.981704
SAT Critical Reading Avg. Score         0.976825
SAT Math Avg. Score                     0.956405
AP Test Takers                          0.572332
Total Exams Taken                       0.561403
Number of Exams with scores 3 4 or 5    0.560564
N_p                                     0.432785
N_s                                     0.428947
total_students                          0.393918
NUMBER OF STUDENTS / SEATS FILLED       0.387976
AVERAGE CLASS SIZE                      0.363332
NUMBER OF SECTIONS                      0.350881
Total Cohort                            0.315132
N_t                                     0.300124
SIZE OF LARGEST CLASS                   0.296141
saf_t_11                                0.293385
aca_s_11                                0.277466
saf_tot_11                              0.274777
saf_s_11                                0.270590
rr_s                

---
# Heat map on a map:
---
<font size=3>
Lets wrap up this data cleaning project with creation of an interactive map... zoom in to get more accurate location plots
</font>

In [35]:
import folium
from folium import plugins

In [49]:
schools_heatmap = folium.Map(location=[full['lat'].mean(), full['lon'].mean()], zoom_start=10)
schools_heatmap.add_children(plugins.HeatMap([[row["lat"], row["lon"]] for name, row in full.iterrows()]))
schools_heatmap.save("heatmap.html")
schools_heatmap

/home/harsh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  


# End Notes on Data Cleaning project:
---
**This notebook was intended to display these skills:**
- Acquire data for analysis.
- Combining different datasets for a more in depth analysis.
- Data cleaning to create a tidy data for analysis by anyone.
- Depending on data at hand, possibilities of analysing and presenting in different formats ( in this case using lattitude and longitude from a random data and using it at our advantage to create a map representation).